## Часть1 

### В этой части предлагается первый способ осуществления  метода одного ближайшего соседа, где будут использованы основные типы данных как списки и кортежи

In [33]:
import numpy as np
from sklearn.datasets import load_digits
from collections import OrderedDict
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
digits = load_digits()

In [3]:
X = digits.data
y = digits.target

**Разбиваем выборку на обучени и тест в соотношении 75:25 соответственно**

In [4]:
X_train = X[:1347]
y_train = y[:1347]

X_test = X[1347:]
y_test = y[1347:]

**Списки evklid, metki понадобятся во второй части, так что здесь на них не обращаем внимания**

In [5]:
evklid = []
metki = []

**Список evklid_metric будет включать в себя список из кортежей. В кортеже будет содержаться пара - растоянние 
от тестового объекта до объекта тренировочной выборки и метка на этом объекте тестовой выборки**

In [6]:
evklid_metric = []

In [7]:
for i in range(len(X_test)):
    for j,k in zip(range(len(X_train)), range(len(y_train))):
        evklid.append(np.sum((X_test[i] - X_train[j])**2))
        metki.append(y_train[k])
        evklid_metric.append((np.sum((X_test[i] - X_train[j])**2), y_train[k]))

**Далее записываем функцию fragmentation_evklid_metric, которая разделит весь большой список evklid_metric на
450 списков, которые в свою очередь будут содежать по 1347 кортежей**

In [8]:
fragmentation_evklid_metric = lambda lst, step: [lst[i:i+step] for i in range(0, len(lst), step)]

In [9]:
no_sorted = fragmentation_evklid_metric(evklid_metric, 1347)

*Далее будм производить сортировку. Но сначала её отработаем на менее объёмном массиве данных, чтобы убедиться, что придуманный код работает исправно и делает то, что нужно*

*Начало*

In [10]:
a = [[(300,1),(600,2), (250,3)], [(500,6), (700,3), (100,2)]]
b = []
for i in a:
    a = sorted(i)
    b.append(min(a)[1]) # достаю метку, соответствущая минимальному расстоянию до тестового объекта

In [11]:
b

[3, 2]

*Конец*

**Применяем теперь это всё к нашим данным**

In [12]:
sorted_list = []
for i in no_sorted:
    sort = sorted(i)
    sorted_list.append(min(sort)[1])

In [13]:
print (sorted_list[0:20])
print (y_test[0:20])

[3, 7, 3, 3, 4, 6, 6, 6, 4, 9, 1, 5, 0, 9, 6, 2, 8, 2, 0, 0]
[3 7 3 3 4 6 6 6 4 9 1 5 0 9 5 2 8 2 0 0]


**Определяем долю ошибок. Либо так...**

In [14]:
score_list = []
for i,j in zip(sorted_list, y_test):
    if i != j:
        score_list.append(1)

answer_1 =   len(score_list)/len(y_test)
answer_1

0.03777777777777778

**...либо так**

In [15]:
score = 1/(len(y_test))*sum([1 if i != j else 0 for i,j in zip(sorted_list, y_test)])
score

0.03777777777777778

## Часть 2

### В этой части предлагается второй способ осуществления метода блжайшего соседа. Основные структуры данных, которые здесь будут использованы - это списки и словари.

**Разбиваем списки evklid и metki на 450 списков, каждый из которых содержит по 1347 элементов**

In [16]:
evklid_new = fragmentation_evklid_metric(evklid, 1347)
metki_new = fragmentation_evklid_metric(metki, 1347)

*Как и в предыдущей части, отработаем сортировку на меньшем объёме данных, чтобы проверить правильность работы кода.*

*Начало*

In [17]:
a = [[1,2,60,0.6,0.9],[2,4,3,0.1,0.5]]
b = [['a','b','e','g','p'],['c','d','l','i','R']]
list_dict = []
for i,j in zip(a,b):
    Dict = {}
    for m,l in zip(i,j):
        Dict.update({m: l})
    list_dict.append(Dict)

In [18]:
list_dict

[{1: 'a', 2: 'b', 60: 'e', 0.6: 'g', 0.9: 'p'},
 {2: 'c', 4: 'd', 3: 'l', 0.1: 'i', 0.5: 'R'}]

In [19]:
list_sort = []

In [20]:
for i in list_dict:
    s = OrderedDict(sorted(i.items(), key=lambda t: t[0]))
    list_sort.append(s)
print (list_sort)

[OrderedDict([(0.6, 'g'), (0.9, 'p'), (1, 'a'), (2, 'b'), (60, 'e')]), OrderedDict([(0.1, 'i'), (0.5, 'R'), (2, 'c'), (3, 'l'), (4, 'd')])]


In [21]:
Metki = []

In [22]:
for i in list_sort:
    Metki.append(list(i.values())[0])

In [23]:
Metki

['g', 'i']

*Конец*

**Теперь приступаем к сортировке наших данных**

In [24]:
Big_list_dict = []
for i,j in zip(evklid_new, metki_new):
    Big_dict = {}
    for distance,metka in zip(i,j):
        Big_dict.update({distance:metka})
    Big_list_dict.append(Big_dict)

In [25]:
list_sort_dicts = []

In [26]:
for i in Big_list_dict:
    sort = OrderedDict(sorted(i.items(), key=lambda t: t[0]))
    list_sort_dicts.append(sort)

**В списке true_metki содержатся все метки минимальных расстояний от объекта из тестовой выборки до объекта обучающе выборки**

In [27]:
true_metki = []

In [28]:
for i in list_sort_dicts:
    true_metki.append(list(i.values())[0])

In [32]:
Score_list = []

for i,j in zip(true_metki, y_test):
    if i != j:
        Score_list.append(1)
answer_1 = len(Score_list)/len(y_test)
print ('Accuracy:', answer_1)

Accuracy: 0.03777777777777778


**Записываем полученнный ответ в файл**

In [23]:
with open('D:\Coursera\KNN_answer_1.txt', 'w') as file:
    file.write(str(answer_1))
file.close()

**Теперь обучаем нашу модель методом случайных лесов и сравниваем полученные доли ошибок на тестовых выборках**

In [33]:
model = RandomForestClassifier(n_estimators = 1000)
model_fit = model.fit(X_train, y_train)

In [34]:
answer_2 = 1- model.score(X_test, y_test)
answer_2

0.06666666666666665

In [27]:
with open('D:\Coursera\KNN_answer_2.txt', 'w') as file:
    file.write(str(answer_2))
file.close()

**Сверяем метод 1NN, написанный своими руками, со встроенным методом путём сравнения доли ошибок**

In [20]:
model = KNeighborsClassifier(n_neighbors=1, weights='distance')
model.fit(X_train, y_train)

y_predicted = model.predict(X_test)
accuracy = accuracy_score(y_test, y_predicted)

In [21]:
1-accuracy

0.0377777777777778